In [29]:
%%writefile reducerfile.py

import sys
import heapq 

li = []

#heapq.heapify(li)

currentWord = None
val = 0

for line in sys.stdin:
    try:
        word, value = line.strip().split('\t',1)
        if(word == currentWord):
            val+=1
        else:
            if currentWord:
                heapq.heappush(li,(val,currentWord))
                if len(li) > 7:
                    heapq.heappop(li)
            currentWord = word
            val = 1
    except:
        continue

heapq.heappush(li,(val,currentWord))
if len(li) > 7:
    heapq.heappop(li)
    
while li:
    value,word = heapq.heappop(li)
    print "%s\t%d" % (word,value)
    

Overwriting reducerfile.py


In [21]:
%%writefile mapperfile.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t',1)
    except valueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print "%s\t%d" % (word.strip().lower(),1)

Overwriting mapperfile.py


In [30]:
%%writefile shellfile.sh

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -files mapperfile.py,reducerfile.py \
    -mapper 'python mapperfile.py' \
    -reducer 'python reducerfile.py' \
    -numReduceTasks 1 \
    -input /data/wiki/en_articles_part/articles-part \
    -output word_frequency3

Overwriting shellfile.sh


In [32]:
!hdfs dfs -ls word_frequency3

Found 2 items
-rw-r--r--   1 jovyan supergroup          0 2019-12-27 19:21 word_frequency3/_SUCCESS
-rw-r--r--   1 jovyan supergroup         71 2019-12-27 19:21 word_frequency3/part-00000


In [31]:
!hdfs dfs -cat word_frequency3/part-00000

is	126420
a	236225
to	241467
in	292354
and	342715
of	447464
the	822164
